In [44]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)


In [45]:
dataDir='/Users/azakaria/Downloads'
#dataType='val2017'
annFile=f'/Users/azakaria/Downloads/nightowls_validation.json'

In [46]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [47]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
nms=[cat['name'] for cat in cats]

In [48]:
catIds = coco.getCatIds()

In [52]:
negative_bbox_annotations = []
imgIds = coco.getImgIds()
def remove_negative_bbox_anns(anns):
    valid_anns = []
    negative_bbox = False
    for ann in anns:
        for bbox_val in ann['bbox']:
            if bbox_val >= 0:
                #This is throwing a keyerror
                if ann['id'] in imgIds:
                    negative_bbox = True
        negative_bbox = False
        valid_anns.append(ann)
    for v in valid_anns:
        print(v['bbox'])
    return valid_anns

from pathlib import Path
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]
def generate_labels(numImgs):
    # using with statement
    imgIds = coco.getImgIds()
    random.shuffle(imgIds)
    imgIds = imgIds[:numImgs]
    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            anns = coco.loadAnns(coco.getAnnIds(imgIds = imgId, catIds=catIds))
            anns = remove_negative_bbox_anns(anns)
            for ann in anns:
                #filter out negative bb
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                """
                print('filename')
                print(filename)
                print("*ann['bbox']")
                print(*ann['bbox'])
                print("img['width'], img['height']")
                print(img['width'], img['height'])
                print('x_center,y_center,width,height')
                print(x_center,y_center,width,height)
                print('--------------')
                """
                if(int(cat_map[ann["category_id"]]) > 2):
                    print(cat_map[ann["category_id"]])
                file.write(f'{cat_map[ann["category_id"]]}\t{x_center}\t{y_center}\t{width}\t{height}\n')
    return


In [54]:
import os
import shutil
from pathlib import Path
import os.path

def copy_images_for_dataset_dir(src_image_dir, dest_image_dir, image_extension):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        #print(src)
        #print(dest)
        #print('----------------')
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
            #print(f'{src} is a file')
        else:
            print(f'{src} is not a file')
    return


#rm -rf dataset/images/*; rm -rf dataset/labels/*
#dataset/labels
generate_labels(120)
#dataset/images
copy_images_for_dataset_dir(Path('/Users/azakaria/Downloads/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/'), '.png')
#docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val

[459, 212, 190, 102]
[662, 211, 23, 79]
3
[306, 221, 41, 105]
[650, 199, 22, 34]
[622, 200, 28, 98]
3
[561, 207, 51, 132]
[621, 220, 47, 123]
[121, 226, 24, 59]
[821, 206, 29, 83]
[120, 185, 52, 193]
[901, 165, 82, 254]
[902, 230, 20, 43]
[945, 220, 21, 71]
3
[28, 261, 183, 117]
3
[991, 254, 28, 103]
[914, 266, 34, 89]
[37, 246, 16, 33]
3
[737, 238, 37, 70]
[378, 228, 24, 50]
[974, 212, 122, 163]
3
[432, 252, 24, 25]
3
[956, 234, 8, 22]
3
[650, 219, 24, 64]
[674, 215, 22, 47]
[-59, 242, 232, 115]
[166, 254, 26, 62]
3
[702, 247, 19, 44]
[652, 238, 34, 47]
3
[362, 209, 48, 147]
[110, 287, 116, 73]
3
[-10, 252, 120, 89]
3
[541, 315, 72, 213]
[611, 356, 74, 73]
3
[999, 229, 46, 122]
[966, 240, 40, 108]
[110, 206, 49, 125]
[160, 223, 37, 87]
3
[139, 215, 43, 157]
[63, 218, 42, 138]
[32, 230, 33, 119]
[84, 246, 34, 101]
[616, 226, 27, 62]
